In [18]:
import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    set_seed,
)
from trainer_seq2seq import Seq2SeqTrainer
import pandas as pd

In [11]:
model_name_or_path = "THUDM/chatglm-6b"
auto_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
# cut_words = auto_tokenizer.tokenize('对比原始的分词和最新的分词器')
# print("cut_words = ",cut_words)
# auto_result04 = auto_tokenizer.convert_tokens_to_ids(cut_words)
# print("auto_result04 = ", auto_result04)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [72]:
cut_words = auto_tokenizer.tokenize('对比原始的分词和最新的分词器')
print("cut_words = ",cut_words)

cut_words =  ['▁', '对比', '原始的', '分', '词', '和', '最新的', '分', '词', '器']


In [22]:
df = pd.read_csv('../../eli/data/test_data/eli_1000_cleaned_marked.csv')
df.head(1)

,题目,考察能力,候选人回答,题目回答1\n请针对下面的题目和候选人的回答，如果题目和回答与{能力名称} 相关，就候选人的{能力名称} 帮我打个分。回答格式请用第一行显示分数，百分制。第二段是简介。如果题目和{能力名称} 无关，请第一行显示“0分”，第二行显示描述“该题目和{能力名称} 无关，无法测评”。面试题目:{面试题目} 候选人回答：{候选人回答内容},题目回答2\n请针对下面的题目和候选人的回答，如果题目和回答与{能力名称} 相关，就候选人的{能力名称} 帮我打个分。回答格式请用第一行显示分数，最高100分，第二段是简介。如果候选人回答中描述了具体的案例则可以酌情加分，否则可以适当减分。如果题目和{能力名称} 无关，请第一行显示“0分”，第二行显示描述“该题目和{能力名称} 无关，无法测评”。面试题目:{面试题目} 候选人回答：{候选人回答内容},题目回答3\n请针对下面的题目和候选人的回答，如果题目和回答与{能力名称}相关，就候选人的{能力名称}帮我打个分。回答格式：用两段阐述，第一段是显示分数，百分制，第二段是用中文对候选人的评价以及对面试官的建议。如果题目和{能力名称}无关，请第一行显示“0分”，第二行显示描述“该题目和{能力名称}无关，无法测评”。面试题目：{题目内容} 候选人回答：{回答内容},回答1 评价标注,回答2 评价标注,准确分数
0,谈一谈你未来的阶段性目标（三个月/半年/一年/三年）,管理能力,", 个人的计划是这样的，在三个月之内，个人目标要做四单买卖，业绩达到10万以上。因为对我个人...",分数：75分\n该候选人在三个月内有具体的目标并且基于自己过往的经验制定了该目标，但是在半年...,60分，候选人回答中有具体的计划和目标，但缺乏具体的策略和实施方法。在管理能力方面，需要进一...,分数：80分\n该候选人具有较强的管理能力，能够设定个人阶段性目标，并有明确的计划去实现这些...,准确,比1 准确,60.0


In [78]:
# 去重操作
df_unique = df.drop_duplicates(subset=['题目'])
df_unique["题目"] = df_unique["题目"].str.strip().str.lstrip(',')

/tmp/ipykernel_87243/1313266864.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["题目"] = df_unique["题目"].str.strip().str.lstrip(',')


In [87]:
def cut_word(row):
    # print(row)
    ret = auto_tokenizer.tokenize(row)
    # print(ret)
    # print('-'*50)
    return ret

In [88]:
# df_unique = df_unique.drop("cuted_words")
df_unique["cuted_words"] = df_unique["题目"].apply(cut_word)

/tmp/ipykernel_87243/1027728201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["cuted_words"] = df_unique["题目"].apply(cut_word)


In [89]:
df_unique["cuted_words"]

0      [▁, 谈一谈, 你, 未来的, 阶段性, 目标, (, 三个月, /, 半年, /, 一年...
1           [▁请, 举, 一个, 为自己, 设立, 目标, ,, 并, 最终实现, 的例子, 。]
3      [▁, 您, 以前, 从事, 过, 哪些, 工作, ?, 您, 工作, 时间最长, 的工作,...
4      [▁How, ▁do, ▁you, ▁normal, y, ▁handle, ▁critic...
5      [▁当你, 与别人, 意见, 产生, 分歧, 的时候, ,, 你是, 如何处理, 的, ?,...
6              [▁请, 举, 一个, 主动, 超越, 职责, 完成, 工作任务, 的例子, 。]
8      [▁, 您, 以前, 是否, 从事, 过, 与, 汽车, 生产, 相关的, 工作, ?, 如...
10     [▁Could, ▁you, ▁tell, ▁me, ▁three, ▁things, ▁y...
13     [▁What, ▁is, ▁the, ▁biggest, ▁challenge, ▁of, ...
16     [▁请, 简单, 描述, 最近, 一份, B, 端, 产品, 经历, ,, 所属, 行业, ...
20     [▁, 请问, 您, 是否, 负责, 过, C, 端, 产品, ?, 如果有, ,, 请, ...
24     [▁What, ▁is, ▁the, ▁biggest, ▁challenge, ▁of, ...
33     [▁What, ▁is, ▁the, ▁biggest, ▁challenge, ▁of, ...
35     [▁What, ▁is, ▁the, ▁biggest, ▁challenge, ▁of, ...
37     [▁What, ▁is, ▁the, ▁biggest, ▁challenge, ▁of, ...
39     [▁你, 压力, 最大的, 阶段, 是什么时候, ?, 当时, 遇到了, 什么情况, ?, ...
40     [▁What, ▁is, ▁the, ▁biggest, ▁challenge, ▁of, ...
43     [▁What, ▁is, ▁the, ▁bigg